In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import shutil
from shutil import copyfile


In [2]:
def check_mask_nearby(i,j,mask,ndist):
    # i = index along i of point to test
    # j = index along j of point to test
    # mask = 2D mask 
    # ndist = distance from center to check
    if (i > (ndist-1)):
        TEST_i = np.nansum(mask[i-ndist:i+ndist+1,j])
    else:
        TEST_i = np.nansum(mask[i:i+ndist+1,j])
    if (j > (ndist-1)):
        TEST_j = np.nansum(mask[i,j-ndist:j+ndist+1])
    else:
        TEST_j = np.nansum(mask[i,j:j+ndist+1])
    return TEST_i, TEST_j    


In [3]:
def find_nearest_sea_point_ij(i,j,mask,ndist,rnf_data):
        arr= np.array([rnf_data[i-ndist,j] + mask[i-ndist,j], \
                    rnf_data[i+ndist,j]    + mask[i+ndist,j],    \
                    rnf_data[i,j-ndist]   + mask[i,j-ndist],   \
                    rnf_data[i,j+ndist]   + mask[i,j+ndist]])
        ind=np.array(np.argwhere(arr==0))
        if len(ind)==1:
            if ind == 0:
                return(i-ndist,j)
            if ind == 1:
                return(i+ndist,j)  
            if ind == 2:
                return(i,j-ndist)      
            if ind == 3:
                return(i,j+ndist)
            return ind
        elif len(ind)>1: # more than one unused sea point around
#             print("problem",len(ind) )
            out=[]
            if arr[0] == 0:
                out=np.append(out,[i-ndist,j])
            if arr[1] == 0:
                out=np.append(out,[i+ndist,j])  
            if arr[2] == 0:
                out=np.append(out,[i,j-ndist])      
            if arr[3] == 0:
                out=np.append(out,[i,j+ndist])
            return out
        else: # Worst case : there is no unused sea point around : return used sea point around
            print("WARNING : point at i,j =",i,j, " has no empty sea point around. Adding rnf value to the nearby sea points")
            out=[]
            if arr[0] < 1:
                out=np.append(out,[i-ndist,j])
            if arr[1] < 1:
                out=np.append(out,[i+ndist,j])  
            if arr[2] < 1:
                out=np.append(out,[i,j-ndist])      
            if arr[3] < 1:
                out=np.append(out,[i,j+ndist])
            return out

In [4]:
def get_npoints(i,j,mask,ndist,rnf_data):
        # return the number of sea points at ndist distance
        arr= np.array([rnf_data[i-ndist,j] + mask[i-ndist,j], \
                    rnf_data[i+ndist,j]    + mask[i+ndist,j],    \
                    rnf_data[i,j-ndist]   + mask[i,j-ndist],   \
                    rnf_data[i,j+ndist]   + mask[i,j+ndist]])
        ind=np.array(np.argwhere(arr==0))
        if len(ind>0):
            return len(ind)
        else:
            ind=np.array(np.argwhere(arr<1))
            return len(ind)



In [7]:
def move_runoffs_ij(rnf_2D,rnf_point,mask,lverb):
# --------------------------------------------------------------------------------------------------
# PURPOSE :
# Check runoff data and move data off the coast or over land at the first available point near the coast
# ------------------------------------
# CASE 1 : at least one earth point in i, j
# ------------------------------------
# CASE 2 : Runoff point is surrounded by sea : 
# Find the nearest coast point
# If more than one coast points are available, average avor the two coast points
#
#
# --------------------------------------------------------------------------------------------------
# Inputs : 
# rnf_2D = runoff data in 2 dimension
# rnf_point = array with indexes of rnf points (should have the form : (2,N), 
# where N is the number of runoff points)
# mask = output mask (1 over sea points and 0 over land points)
# lverb = verbose (True or False)
# --------------------------------------------------------------------------------------------------
# Output : New 2D array with moved runoffs
# --------------------------------------------------------------------------------------------------

# Initialisation
    nseuil=30 # over 30 points distance, do not replace runoff
    rnf_2D_new = np.copy(rnf_2D)
    mask_data=mask
    mask_data=np.where(mask_data==1,0,1) # invert mask
    sea_point_not_found=0.
    ndist=1
    
# --------------------------------------------------------------------------------------------------
# BEGIN LOOP

# Loop over valid runoff values only  
    for n in range(len(rnf_point[0,:])): 
        N_earth_points_i, N_earth_points_j = check_mask_nearby(rnf_point[0,n],\
                                                               rnf_point[1,n],mask_data,ndist)
        N_earth_points_ij = N_earth_points_i + N_earth_points_j
        
# CASE 1 : POINT IS OVER LAND ------------
        if(N_earth_points_ij > 0): # at least one earth point nearby
            if (mask_data[rnf_point[0,n],rnf_point[1,n]]==1):
                if lverb: print("==> j, i = ", rnf_point[0,n],rnf_point[1,n],                    \
                                "is on mask point with", N_earth_points_i -1                     \
                      ,"earth points along i nearby and", N_earth_points_j - 1,                  \
                     "points along j")
                    
    # CASE 1.1 : POINT IS SURROUNDED BY EARTH ------------
            # Loop until we find at least one sea point in ndist perimeter
                if N_earth_points_ij == 6: # Surrounded by earth
                    print("WARNING !!! : point is surrounded by earth at i,j =", rnf_point[0,n],rnf_point[1,n])

                        
    # CASE 1.2 : AT LEAST ON SEA POINT in ndist=1 perimeter ------------
                else: # 
                    ind = find_nearest_sea_point_ij(rnf_point[0,n],rnf_point[1,n],mask_data,        \
                                                 ndist,rnf_2D.values)
                    npoints = get_npoints(rnf_point[0,n],rnf_point[1,n],mask_data,ndist,         \
                                          rnf_2D.values)               
                # Now replace data
                if npoints == 0: 
                    # No available points nearby without an already existing runoff value => avoid
                    rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] = 0.  
                    print(rnf_point[0,n],rnf_point[1,n],"ISSUE AT THIS POINT !!")
                if npoints == 1: 
                    rnf_2D_new[int(ind[0]),int(ind[1])] = rnf_2D_new[int(ind[0]),int(ind[1])]    \
                    + rnf_2D_new[rnf_point[0,n],rnf_point[1,n]]
                    rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] = 0.   
                    if lverb: print("moving point j,i = ", rnf_point[0,n],rnf_point[1,n],        \
                                    "at point j,i = ", ind)
                if npoints == 2: 
                    rnf_2D_new[int(ind[0]),int(ind[1])] = rnf_2D_new[int(ind[0]),int(ind[1])]    \
                    + (rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] / 2) 
                    rnf_2D_new[int(ind[2]),int(ind[3])] = rnf_2D_new[int(ind[2]),int(ind[3])]    \
                    + (rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] / 2 )
                    rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] = 0.
                    if lverb: print("dispatch point j,i = ", rnf_point[0,n],rnf_point[1,n],      \
                                    "at 2 points j1,i1,j2,i2 = ", ind)
                if npoints == 3: 
                    rnf_2D_new[int(ind[0]),int(ind[1])] = rnf_2D_new[int(ind[0]),int(ind[1])]     \
                    + (rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] / 3 )
                    rnf_2D_new[int(ind[2]),int(ind[3])] = rnf_2D_new[int(ind[2]),int(ind[3])]     \
                    + (rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] / 3) 
                    rnf_2D_new[int(ind[3]),int(ind[4])] = rnf_2D_new[int(ind[3]),int(ind[4])]     \
                    + (rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] / 3) 
                    rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] = 0.
                    if lverb: print("moving point j,i = ", rnf_point[0,n],rnf_point[1,n],         \
                                    "at 3 points j1,i1,j2,i2,j3,i3 = ", ind)

                                        
                if lverb: print("..........")

# CASE 2 : POINT IS SURROUNDED BY SEA ------------
        else: # no earth point nearby

            if lverb: print(rnf_point[0,n],rnf_point[1,n], "is in the middle of the sea",        \
                            N_earth_points_i, N_earth_points_j,mask_data[rnf_point[0,n]+1,rnf_point[1,n]])
            # ------------------------------------------------
            # Find nearest coast point from 1 to nseuil distance
            ind = find_nearest_coast_point(rnf_point[0,n],rnf_point[1,n],mask_data,rnf_2D.values,nseuil)
            npoints=len(ind)/2
            # ------------------------------------------------
                      
            if npoints == 0: # No available points nearby without an already existing runoff value => avoid
                rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] = 0.  
                print("WARNING : point at i, j = ", rnf_point[0,n],rnf_point[1,n] ,              \
                      " is way too far from coast => ignore")
            if npoints == 1: 
                rnf_2D_new[int(ind[0]),int(ind[1])] = rnf_2D_new[int(ind[0]),int(ind[1])]        \
                + rnf_2D_new[rnf_point[0,n],rnf_point[1,n]]
                rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] = 0.   
                if lverb: print("moving point j,i = ", rnf_point[0,n],rnf_point[1,n],            \
                                "at point j,i = ", ind)
            if npoints == 2: 
                rnf_2D_new[int(ind[0]),int(ind[1])] = rnf_2D_new[int(ind[0]),int(ind[1])]        \
                + (rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] / 2) 
                rnf_2D_new[int(ind[2]),int(ind[3])] = rnf_2D_new[int(ind[2]),int(ind[3])]        \
                + (rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] / 2 )
                rnf_2D_new[rnf_point[0,n],rnf_point[1,n]] = 0. 
                if lverb: print("dispatch point j,i = ", rnf_point[0,n],rnf_point[1,n],\
                                "at 2 points j1,i1,j2,i2 = ", ind)
            if npoints > 2: print("OVER 2")
            if lverb: print("..........")

        sea_point_not_found=0.
        ndist=1
        # Next point
    return rnf_2D_new
# --------------------------------------------------------------------------------------------------
# END LOOP

In [8]:
year=2017

rnf_inp_file='/data/rd_exchange/tbrivoal/RUNOFFS_eNEATL36/runoff_eNEATL36_with_BDY_y'+str(year)+'.nc'
inp_domain_cfg_file='/data/rd_exchange/tbrivoal/RUNOFFS_eNEATL36/domain_cfg.nc'
out_runoff_file='/data/rd_exchange/tbrivoal/RUNOFFS_eNEATL36/runoff_eNEATL36_with_BDY_y'+str(year)+'.nc'
mask_file='/data/rd_exchange/tbrivoal/RUNOFFS_eNEATL36/mesh_mask.nc'
output_folder="/data/rd_exchange/tbrivoal/RUNOFFS_eNEATL36/"


In [9]:
# Open files
inp_rnf_2D=xr.open_dataset(rnf_inp_file)
inp_domain_cfg=xr.open_dataset(inp_domain_cfg_file,drop_variables={"x", "y",})
maskf=xr.open_dataset(mask_file,drop_variables={"x", "y",})


In [10]:
# Open variables
# mask_inp =  inp_domain_cfg.top_level.squeeze()
mask_inp =  inp_domain_cfg.top_level.squeeze()

# mask_inp = maskf.tmask[0,0,:,:].squeeze()
rnf_2D=inp_rnf_2D.orca_costal[:,:,:]
rnf_2D_new=np.copy(rnf_2D)

In [11]:
rnf_point = np.array(np.where(rnf_2D[0,:,:].values>0)) # Assuming runoffs does not move

for i in range(len(rnf_2D[:,0,0])):
    print(i)
    rnf_2D_new[i,:,:]=move_runoffs(rnf_2D[i,:,:],rnf_point,mask_inp,False)

0
1
2
3
4
5
6


7
8
9
10
11
12


13
14
15
16
17
18


19
20
21
22
23
24


25
26
27
28
29
30


...

337
338
339
340
341
342


343
344
345
346
347
348


349
350
351
352
353
354


355
356
357
358
359
360


361
362
363
364


In [12]:
year=2017 # year of runoff file

# Date variables ----------------------------------------------------------------------------------------
dstart_year= str(year) + "-01-01" # 1st day of the year
dend_year= str(year) + "-12-31" # last day of the year

dstart_clim_monthly = str(year) + "-01-15" # for 12 month clim
dend_clim_monthly = str(year) + "-12-15" # for 12 month clim
dstart_minus_1_month = str(year-1) + "-12-01" # used for conversion from monthly to daily clim data

############################ Save in a Netcdf ###################################
time_counter_daily = pd.date_range(dstart_year,dend_year, freq="D") # time dimension

# time_counter_daily_float = pd.to_numeric(time_counter_daily, downcast='float')
#time_counter_daily.values.astype("float")
# Create xarray dataarray
rnf_2D_daily_new_da = xr.DataArray(
    data=rnf_2D_new,
    dims=["time_counter", "y", "x"],
    coords=dict(time_counter=time_counter_daily.values),
    attrs=rnf_2D.attrs,
    name=rnf_2D.name,
     )



# Create dataset
ds = xr.merge([rnf_2D_daily_new_da, inp_rnf_2D.nav_lat, inp_rnf_2D.nav_lon])
ds.attrs=inp_rnf_2D.attrs
# Save to netcdf
outfile =   out_domain_cfg_file
# outfile = output_folder + "runoff_eNEATL36_with_BDY_y" + str(year) + '.nc'  

ds.to_netcdf(outfile, mode="w")

In [17]:
print(np.nansum(rnf_2D_new[:,:,:]) - np.nansum(inp_rnf_2D.orca_costal[:,:,:]))

-1.6370904631912708e-11
